# Load Datasets

https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset?select=Ratings.csv
https://www.kaggle.com/datasets/arashnic/book-recommendation-dataset?select=Books.csv

In [1]:
# load datasets
import pandas as pd
books = pd.read_csv("books datasets/Books.csv")
ratings = pd.read_csv('books datasets//Ratings.csv')

C:\Users\Noor Saeed\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\Noor Saeed\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (
C:\Users\Noor Saeed\AppData\Local\Temp\ipykernel_21876\3521914861.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("books datasets/Books.csv")


In [5]:
# Merging both dataset
df = pd.merge(books, ratings, on='ISBN', how='left')
df.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,User-ID,Book-Rating
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,2.0,0.0
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,8.0,5.0
2,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11400.0,0.0
3,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,11676.0,8.0
4,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,41385.0,0.0


# Data Cleaning

In [6]:
df.isnull().sum()

ISBN                      0
Book-Title                0
Book-Author               2
Year-Of-Publication       0
Publisher                 2
Image-URL-S               0
Image-URL-M               0
Image-URL-L               4
User-ID                1209
Book-Rating            1209
dtype: int64

In [7]:
df.dropna(inplace=True)

In [9]:
df = df.sample(n=20000, random_state=42).reset_index(drop=True)

In [11]:
df.to_csv('books datasets/final_df.csv')

# Popularity Base Recommendation System

To implement a popularity-based recommendation system using the dataset you provided, the idea is to recommend the most popular books based on the number of ratings or the average rating score. Here’s how you can do this:


Step 1: Group the dataset by Book-Title and calculate the average rating and the number of ratings for each book.

Step 2: Sort the books based on either the average rating or the number of ratings (whichever you define as "popularity").

Step 3: Recommend books that have the highest popularity score.

In [24]:
# Step 1: Calculate average ratings and the number of ratings
book_ratings = df.groupby('Book-Title').agg({'Book-Rating': ['mean', 'count']}).reset_index()

# Step 2: Sort the books based on the count of ratings (popularity) or average rating
book_ratings.columns = ['Book-Title', 'Avg-Rating', 'Num-Ratings']
book_ratings_sorted = book_ratings.sort_values(by=['Num-Ratings', 'Avg-Rating'], ascending=False)

# Step 3: Recommend the top N popular books
top_books = book_ratings_sorted.head(10)  # Change N to the number of books you want to recommend

# Step 4: Merge with the original dataset to include Author, Publisher, and Image-URL-M
top_books_final = pd.merge(top_books, df[['Book-Title','Image-URL-M']], on='Book-Title', how='left')

# Step 5: Drop duplicate rows based on Book-Title
top_books_final = top_books_final.drop_duplicates(subset=['Book-Title'])

# Display the top N recommended books with the additional columns
top_books_final 

,Book-Title,Avg-Rating,Num-Ratings,Image-URL-M
0,Wild Animus,1.063492,63,http://images.amazon.com/images/P/0971880107.0...
63,The Lovely Bones: A Novel,4.000000,29,http://images.amazon.com/images/P/0316666343.0...
92,Divine Secrets of the Ya-Ya Sisterhood: A Novel,4.181818,22,http://images.amazon.com/images/P/0060928336.0...
114,The Da Vinci Code,4.045455,22,http://images.amazon.com/images/P/0385504209.0...
136,The Nanny Diaries: A Novel,3.952381,21,http://images.amazon.com/images/P/0312291639.0...
157,Summer Sisters,3.294118,17,http://images.amazon.com/images/P/0440226430.0...
174,The Five People You Meet in Heaven,5.062500,16,http://images.amazon.com/images/P/0786868716.0...
190,Suzanne's Diary for Nicholas,4.062500,16,http://images.amazon.com/images/P/0446679593.0...
206,Me Talk Pretty One Day,3.800000,15,http://images.amazon.com/images/P/0316777722.0...
221,A Painted House,2.533333,15,http://images.amazon.com/images/P/038550120X.0...


In [ ]:
top_books_final.to_csv("books datasets/top_books_final.csv")

# Search Base Recommendation System

In [34]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv("books datasets/final_df.csv")

# Drop duplicates based on 'Book-Title'
df = df.drop_duplicates(subset=['Book-Title']).reset_index(drop=True)

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['Book-Title'])  # Recompute TF-IDF after dropping duplicates

In [35]:
from sklearn.metrics.pairwise import cosine_similarity

def search_based_recommendation(book_title, df,tfidf, tfidf_matrix, top_n=10):
    """
    Recommend books based on similarity of titles using TF-IDF and cosine similarity.
    
    Parameters:
        book_title (str): The title of the book to search for recommendations.
        df (pd.DataFrame): The dataset containing book information.
        tfidf_matrix (sparse matrix): Precomputed TF-IDF matrix for book titles.
        top_n (int): Number of recommendations to return.
    
    Returns:
        pd.DataFrame: Recommended books (title, rating, image URL).
    """
   

    # Transform the input title to match TF-IDF dimensions
    query_vector = tfidf.transform([book_title])
    
    # Compute cosine similarity between the query and all book titles
    cosine_sim = cosine_similarity(query_vector, tfidf_matrix).flatten()
    
    # Get indices of the top N most similar books (excluding the input book itself)
    sim_scores = list(enumerate(cosine_sim))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)  # Sort by similarity
    sim_scores = sim_scores[1:top_n + 1]  # Exclude the input book itself
    
    # Get the indices of the recommended books
    book_indices = [i[0] for i in sim_scores]
    
    # Return the relevant columns for the recommended books
    return df[['Book-Title', 'Book-Rating', 'Image-URL-M']].iloc[book_indices].reset_index(drop=True)

In [36]:
# Example usage
book_to_search = "The Fountainhead"  # Replace with the title you want to search
recommended_books = search_based_recommendation(book_to_search, df,tfidf, tfidf_matrix)

recommended_books

,Book-Title,Book-Rating,Image-URL-M
0,The Book of the Crime,7.0,http://images.amazon.com/images/P/0553238116.0...
1,The Novel,8.0,http://images.amazon.com/images/P/0449221431.0...
2,"The Return of the King (The Lord of the Rings,...",0.0,http://images.amazon.com/images/P/0345339738.0...
3,The Return of the Shadow (The History of The L...,7.0,http://images.amazon.com/images/P/0261102249.0...
4,The Book of the Sun,10.0,http://images.amazon.com/images/P/0762401400.0...
5,The War of the Ring (The History of The Lord o...,8.0,http://images.amazon.com/images/P/0618083596.0...
6,"The Lion, the Witch, and the Wardrobe (The Chr...",0.0,http://images.amazon.com/images/P/0064471047.0...
7,"The Lion, the Witch, and the Wardrobe",0.0,http://images.amazon.com/images/P/0590254766.0...
8,The Fellowship of the Ring (The Lord of the Ri...,10.0,http://images.amazon.com/images/P/0618129030.0...
9,The Hobbit and The Lord of the Rings,10.0,http://images.amazon.com/images/P/0345340426.0...


In [37]:
# Example usage
book_to_search = "Pinocchio"  # Replace with the title you want to search
recommended_books = search_based_recommendation(book_to_search, df,tfidf, tfidf_matrix)

recommended_books

,Book-Title,Book-Rating,Image-URL-M
0,The Adventures of Pinocchio: The Story of a Pu...,0.0,http://images.amazon.com/images/P/0968876803.0...
1,Walt Disney's Pinocchio and His Puppet Show Ad...,0.0,http://images.amazon.com/images/P/0394826264.0...
2,Beware the Naked Man Who Offers You His Shirt,5.0,http://images.amazon.com/images/P/0688092292.0...
3,"Night Moves (Tom Clancy's Net Force, No. 3)",0.0,http://images.amazon.com/images/P/042517400X.0...
4,The Fountainhead,10.0,http://images.amazon.com/images/P/0451191153.0...
5,The One-Room Schoolhouse: Stories About the Bo...,0.0,http://images.amazon.com/images/P/0679747699.0...
6,The Tin Drum (Everyman's Library Series),0.0,http://images.amazon.com/images/P/0679420339.0...
7,Still Life With Woodpecker,0.0,http://images.amazon.com/images/P/0553148923.0...
8,"Wings of Fear (Nancy Drew Files, Case No 13)",7.0,http://images.amazon.com/images/P/0671641379.0...
9,Your Mental Health: A Layman's Guide to the Ps...,7.0,http://images.amazon.com/images/P/068483720X.0...


In [38]:
# save files for search base recommendation
import pickle

pickle.dump(tfidf,open("books datasets/tfidf.pkl",'wb'))
pickle.dump(tfidf_matrix, open("books datasets/tfidf_matrix.pkl",'wb'))